In [2]:
import pandas as pd
from glob import glob
import numpy as np

In [3]:
sht_apd = pd.read_csv('./Raw/shootings-append.csv', low_memory=False)

In [4]:
april_files = glob('./Raw/raw dump April2016/*All*')
april_files

['./Raw/raw dump April2016/All complaints during 2013.xls',
 './Raw/raw dump April2016/All complaints during 2012.xls',
 './Raw/raw dump April2016/All complaints during 2014.xls',
 './Raw/raw dump April2016/All complaints during 2016YTD.xls',
 './Raw/raw dump April2016/All complaints during 2015.xls']

In [5]:
df_list = [pd.read_excel(file) for file in april_files]
april_df = pd.concat(df_list, ignore_index=True)

In [10]:
may_df = pd.read_csv('./Clean/May2016/dat_may2016.csv', low_memory=False)

In [11]:
april_complaints = april_df.Complaint_Number.dropna().unique()
may_complaints = may_df.Complaint_Number.dropna().unique()
print(len(april_complaints))
print(len(may_complaints))

7175
361


In [12]:
non_intersect = [complaint for complaint in may_complaints
                 if complaint not in april_complaints]

In [13]:
len(non_intersect)

200

# Cleaning / Combining April and May

In [14]:
clean_april = april_df.copy()
clean_may = may_df.copy()

In [15]:
april_cols = [col.lower().replace(' ', '_') for col in clean_april.columns]
may_cols = [col.lower().replace(' ', '_') for col in clean_may.columns]
clean_april.columns = april_cols
clean_may.columns = may_cols

In April, several of there are multiple fname columns due to misspellings of the column headers. These can be combined, as there are no cases where one column has conflicting information with the other column

In [16]:
# confirming there are no conflicting values
sub_df = clean_april[['accused_officer_fname', 'accusedofficer_fname']]
sub_df = sub_df[(~sub_df.accused_officer_fname.isnull()) & 
                 (~sub_df.accusedofficer_fname.isnull())]
print(sum(sub_df.accused_officer_fname != sub_df.accusedofficer_fname))

sub_df = clean_april[['accused_officer_fname', 'accsued_officer_fname']]
sub_df = sub_df[(~sub_df.accused_officer_fname.isnull()) & 
                (~sub_df.accsued_officer_fname.isnull())]
print(sum(sub_df.accused_officer_fname != sub_df.accsued_officer_fname))

sub_df = clean_april[['accsued_officer_fname', 'accusedofficer_fname']]
sub_df = sub_df[(~sub_df.accsued_officer_fname.isnull()) & 
                (~sub_df.accusedofficer_fname.isnull())]
print(sum(sub_df.accsued_officer_fname != sub_df.accusedofficer_fname))

0
0
0


In [17]:
clean_april.accused_officer_fname.fillna(clean_april.accsued_officer_fname, inplace=True)
clean_april.accused_officer_fname.fillna(clean_april.accusedofficer_fname, inplace=True)
clean_april.drop(['accsued_officer_fname', 'accusedofficer_fname'], axis=1, inplace=True)

In [18]:
april_cols = clean_april.columns.tolist()
april_diff = sorted([col for col in april_cols if col not in may_cols])
may_diff = sorted([col for col in may_cols if col not in april_cols])

for i in range(len(may_diff)):
    try: 
        print('{:40}{}'.format(april_diff[i], may_diff[i]))
    except IndexError:
        print('{:40}{}'.format('NA', may_diff[i]))

accused_officer_fname                   accused_assignment
accused_officer_lname                   accused_fname
accused_star                            accused_lname
accused_unit                            accused_star_no
discipline_code                         closedatipra_datetime
involved_officer_age                    complaint_date
involved_officer_detail                 converted_rec_i
involved_officer_fname                  cr_required
involved_officer_lname                  fileyr
involved_officer_position               finding_id
involved_officer_race                   iad_ops
involved_officer_sex                    invovled_party_age
involved_officer_unit                   invovled_party_assignment
ipra_closed_date                        invovled_party_description
notification_date                       invovled_party_detail
recommended_number_of_days              invovled_party_fname
NA                                      invovled_party_lname
NA                            

In [19]:
clean_may.columns = [col.replace('party', 'officer').replace('invovled', 'involved')
                     for col in clean_may.columns]
may_cols = clean_may.columns

In [20]:
april_diff = sorted([col for col in april_cols if col not in may_cols])
may_diff = sorted([col for col in may_cols if col not in april_cols])

for i in range(len(may_diff)):
    try: 
        print('{:40}{}'.format(april_diff[i], may_diff[i]))
    except IndexError:
        print('{:40}{}'.format('NA', may_diff[i]))

accused_officer_fname                   accused_assignment
accused_officer_lname                   accused_fname
accused_star                            accused_lname
accused_unit                            accused_star_no
discipline_code                         closedatipra_datetime
involved_officer_unit                   complaint_date
ipra_closed_date                        converted_rec_i
notification_date                       cr_required
recommended_number_of_days              fileyr
NA                                      finding_id
NA                                      iad_ops
NA                                      involved_officer_assignment
NA                                      involved_officer_description
NA                                      involved_officer_type
NA                                      ipra_assign_date_time
NA                                      ipra_investigate_begin_date
NA                                      ipra_investigate_end_date
NA         

In [21]:
# these two columns seems to be the same as well
print(clean_may.penalty_code.unique())
print(clean_april.discipline_code.unique())

[nan 'VIOLATION NOTED' 'REPRIMAND' 'SUSPENSION' 'SEPARATION']
[nan 'SEPARATION' 'SUSPENSION' 'VIOLATION NOTED' 'REPRIMAND']


In [22]:
# these two columns seems to be the same as well
print(sorted(clean_may.involved_officer_assignment.unique()))
print(sorted(clean_april.involved_officer_unit.unique()))

[2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 20.0, 21.0, 23.0, 45.0, 50.0, 57.0, 116.0, 119.0, 120.0, 121.0, 123.0, 124.0, 126.0, 127.0, 140.0, nan, 1.0, 19.0, 22.0, 24.0, 25.0, 44.0, 113.0, 141.0, 142.0, 151.0, 152.0, 153.0, 180.0, 188.0, 189.0, 191.0, 193.0, 211.0, 212.0, 213.0, 214.0, 215.0, 216.0, 253.0, 277.0, 311.0, 312.0, 313.0, 314.0, 315.0, 341.0, 353.0, 384.0, 393.0, 477.0, 601.0, 603.0, 606.0, 610.0, 620.0, 630.0, 640.0, 701.0, 704.0]
[nan, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 22.0, 24.0, 25.0, 26.0, 44.0, 45.0, 50.0, 51.0, 59.0, 79.0, 86.0, 113.0, 116.0, 119.0, 121.0, 123.0, 124.0, 126.0, 127.0, 135.0, 140.0, 141.0, 142.0, 145.0, 153.0, 163.0, 166.0, 171.0, 177.0, 180.0, 189.0, 191.0, 192.0, 193.0, 196.0, 211.0, 212.0, 213.0, 214.0, 215.0, 261.0, 277.0, 311.0, 312.0, 313.0, 314.0, 315.0, 341.0, 353.0, 377.0, 384.0, 393.0, 441.0, 477.0, 606.0, 608.0, 61

In [23]:
# these two columns seems to be the same as well
print(sorted(clean_may.accused_assignment.unique()))
print(sorted(clean_april.accused_unit.unique()))

[nan, 1.0, 2.0, 4.0, 6.0, 7.0, 8.0, 10.0, 11.0, 18.0, 171.0, 177.0, 189.0, 312.0, 610.0, 620.0]
[nan, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 24.0, 25.0, 26.0, 44.0, 45.0, 50.0, 51.0, 55.0, 59.0, 111.0, 121.0, 123.0, 124.0, 126.0, 141.0, 145.0, 163.0, 166.0, 167.0, 171.0, 177.0, 180.0, 184.0, 189.0, 191.0, 192.0, 193.0, 196.0, 214.0, 277.0, 311.0, 312.0, 313.0, 314.0, 315.0, 341.0, 353.0, 376.0, 377.0, 384.0, 393.0, 442.0, 542.0, 603.0, 606.0, 608.0, 610.0, 620.0, 630.0, 640.0, 650.0, 701.0, 704.0]


In [24]:
replacements = {'accused_fname': 'accused_officer_fname',
                'accused_lname': 'accused_officer_lname',
                'accused_star_no': 'accused_star',
                'no_of_days': 'recommended_number_of_days', 
                'closedatipra_datetime': 'ipra_closed_date',
                'penalty_code': 'discipline_code', 
                'involved_officer_assignment': 'involved_officer_unit',
                'accused_assignment': 'accused_unit'}

clean_may.rename(columns = replacements, inplace=True)

In [25]:
may_cols = clean_may.columns.tolist()
april_diff = sorted([col for col in april_cols if col not in may_cols])
may_diff = sorted([col for col in may_cols if col not in april_cols])

for i in range(len(may_diff)):
    try: 
        print('{:40}{}'.format(april_diff[i], may_diff[i]))
    except IndexError:
        print('{:40}{}'.format('NA', may_diff[i]))

notification_date                       complaint_date
NA                                      converted_rec_i
NA                                      cr_required
NA                                      fileyr
NA                                      finding_id
NA                                      iad_ops
NA                                      involved_officer_description
NA                                      involved_officer_type
NA                                      ipra_assign_date_time
NA                                      ipra_investigate_begin_date
NA                                      ipra_investigate_end_date
NA                                      ipra_investigator_type
NA                                      penalty_status
NA                                      police_shooting
NA                                      police_shooting_no
NA                                      report_status
NA                                      source


#### At this point it's unclear to me what column in the May dataset the "notification_date" column in the April dataset might correspond to

#### As far as I can tell, "accused_unit" and "involved_officer_unit" are not the same

In [26]:
# there are cases where the accused_unit and involved_officer_unit are both not nan, 
# but are also not equal
sub_df = clean_april[['accused_unit', 'involved_officer_unit']]
sub_df = sub_df[(~sub_df.accused_unit.isnull()) & 
                 (~sub_df.involved_officer_unit.isnull())]
print(sum(sub_df.accused_unit != sub_df.involved_officer_unit))

sub_df = clean_may[['accused_unit', 'involved_officer_unit']]
sub_df2 = sub_df[(~sub_df.accused_unit.isnull()) & 
                 (~sub_df.involved_officer_unit.isnull())]
print(sum(sub_df2.accused_unit != sub_df2.involved_officer_unit))

9141
1527


In [27]:
combined_df = pd.concat([clean_april, clean_may], ignore_index=True)

# Subset from 2016-ipra website

In [28]:
complaint_ix = pd.read_excel('/home/dan/Downloads/2016-06-03-complaint-index.xlsx')
doc_ix = pd.read_excel('/home/dan/Downloads/2016-06-03-document-index.xlsx')                

complaint_ix.rename(columns={'COMPLAINT_NUMBER': 'complaint_number'}, inplace=True)
doc_ix.rename(columns={'COMPLAINT_NUMBER': 'complaint_number'}, inplace=True)

In [29]:
ipra_df = pd.merge(combined_df, complaint_ix, on='complaint_number', how='right')
ipra_df = pd.merge(ipra_df, doc_ix, on='complaint_number', how='left')

In [32]:
print(len(ipra_df.complaint_number.unique()))

102


### Ultimately we're going to want to look up CRIDs and get all the information for that incident (right?)

In [36]:
cols = [col for col in ipra_df.columns if col != 'complaint_number']
group = ipra_df.groupby('complaint_number')
df_list = [group[col].unique().reset_index() for col in cols]

In [37]:
summary_df = df_list[0]
for col_df in df_list:
    summary_df = pd.merge(summary_df, col_df, on='complaint_number')

In [40]:
#e.g.
summary_df.ix[1]

complaint_number                                                      1.04253e+06
accused_appointment_date_x                                                  [nan]
accused_appointment_date_y                                                  [nan]
accused_detail                                                              [nan]
accused_officer_fname                                                       [nan]
accused_officer_lname                                                       [nan]
accused_position                                                            [nan]
accused_star                                                                [nan]
accused_unit                                                                [nan]
address                                                                  [1120.0]
beat                                                                      [724.0]
complaint_date                                                         [1/7/2011]
converted_rec_i 

In [46]:
summary_df.to_csv('./summary_ipra.csv')

# Exploring Data

In [41]:
combined_df['full_name'] = combined_df.accused_officer_fname + \
                           '_' + combined_df.accused_officer_lname

In [42]:
officer_complaints = combined_df.groupby('full_name').complaint_number.unique()\
                                .reset_index()

In [43]:
officer_complaints['complaint_counts'] = officer_complaints.complaint_number\
                                                           .apply(lambda x: len(x))

In [44]:
officer_complaints.sort_values(by='complaint_counts', ascending=False).head()

,full_name,complaint_number,complaint_counts
165,ANTHONY_ROSEN,"[1062800, 1066667, 1053667, 1053730, 1055766, ...",10
1255,JESSE_RODRIGUEZ,"[1061127, 1064485, 1053629, 1056779, 1070318, ...",9
2481,ROBERT_LOBIANCO,"[1063527, 1063935, 1066371, 1054352, 1054394, ...",8
2615,SAMUEL_TRUESDALE,"[1061304, 1061698, 1062683, 1064009, 1054999, ...",8
2641,SEAN_CAMPBELL,"[1061897, 1066578, 1054138, 1054349, 1055884, ...",8


In [214]:
officer_complaints.to_csv('./officer_complaints.csv')

In [45]:
complaint_nums_off = combined_df.groupby('complaint_number').full_name.unique()\
                                .reset_index()
complaint_nums_off['counts'] = complaint_nums_off.full_name\
                                                 .apply(lambda x: len(x))

complaint_nums_off.sort_values(by='counts', ascending=False).head()

,complaint_number,full_name,counts
1085,1055981,"[COLIN_MACNIFF, CORY_PETRACCO, DANIEL_FELICIAN...",22
4953,1070415,"[nan, ABDULLAH_BEYAH, ADAM_PURICELLI, BRIAN_MC...",16
825,1055068,"[ADOLFO_GARCIA, ANNE_ZAMZOW, BENJAMEN_FERN, CH...",14
4074,1067139,"[nan, BRIAN_BLACKMAN, DANIEL_SAKO, DARRELL_DAV...",13
4058,1067084,"[nan, ANTHONY_ROSEN, EDWARD_LEIGHTON, ERNEST_C...",12
